<h1>fmap (Frequence Mapper)</h1>

<img align="right" src="docs/assets/cover.png" style="width: 70%; left:30px">

<p align="center">(🇮🇹) Monitora la qualità della ricezione dei segnali radio nelle province siciliane. </p>
<p align="center">(🇺🇸) Monitors the quality of radio signal reception in the Sicilian provinces. </p>

<table>
    <tr>
        <td>
            Coded by <a href="https://github.com/LemuelPuglisi">Lemuel Puglisi</a> e <a href="https://github.com/Gigi-G">Luigi Seminara</a>.
        </td>
        <td>
            <a href="https://github.com/triglie/fmap"></a>
        </td>
    </tr>
</table>